In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 25 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

11 April 2023 ; 13:29:47


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,1,1
1,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1
2,Brook Lopez,C,34,MIL,78,78,30.4,6.1,11.5,0.531,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0
3,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,0
4,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Nikola Jokić,DEN,4.5,1.3,0.7,110,4.515942,1,0.163665
1,Jimmy Butler,MIA,2.0,1.8,0.3,112,3.715625,7,0.135294
2,Giannis Antetokounmpo,MIL,2.7,0.8,0.8,108,4.815873,1,0.124024
3,Draymond Green,GSW,2.6,1.0,0.8,112,3.482192,5,0.105429
4,Jaren Jackson Jr.,MEM,2.0,1.0,3.0,105,4.946032,2,0.103081


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)